In [1]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
%matplotlib notebook
import importlib
from matplotlib import pyplot as plt

In [2]:
with open('C:/Users/ehajizad/lava-nc/neuromorphic-continual-learning/tutorials/coil20_labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle=True)
with open('C:/Users/ehajizad/lava-nc/neuromorphic-continual-learning/tutorials/strided_coil20.npy', 'rb') as f:
    stride_filtered = np.load(f, allow_pickle=True)

In [3]:
scaled_input = 100 * stride_filtered
scaled_input = scaled_input.reshape((scaled_input.shape[0], stride_filtered.shape[1],-1))
print(scaled_input.shape)
flat_input=scaled_input.reshape((scaled_input.shape[0], -1))
print(flat_input.shape)

(1440, 8, 441)
(1440, 3528)


In [4]:
# Bound the feature to the [0,200] range simulating the rate code range
vmax=350
vmin=30
noise_max=100
bounded = scaled_input.copy()
bounded[bounded<vmin]=0  # below 10, no spike
bounded[bounded>vmax]=vmax # above 200, rate code saturates

# Calculate the noise value for each patch
sum_per_patch = bounded.sum(1)
noise = noise_max - sum_per_patch
noise[noise<0]=0 # also bounded in the range of [0,200]

# Append this noise feature to our feature vectors
bounded=np.append(bounded, noise[:,np.newaxis, :], axis=1)
print(bounded.shape) # new noise feature is now added

flat_bounded=bounded.reshape((bounded.shape[0], bounded.shape[1]*bounded.shape[2]))
print(flat_bounded.shape)

(1440, 9, 441)
(1440, 3969)


In [8]:
clf = make_pipeline(StandardScaler(), svm.LinearSVC(max_iter=500))
# clf = svm.LinearSVC(max_iter=10000)
X = flat_input
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test


In [9]:
clf.fit(X_train, y_train)

c:\users\ehajizad\lava-nc\venv\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(max_iter=500))])

In [10]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

y_pred_train=clf.predict(X_train)
y_pred_test=clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Train Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Test Accuracy:",metrics.accuracy_score(y_test, y_pred_test))

Train Accuracy: 1.0
Test Accuracy: 0.7916666666666666
